In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import scipy
import sympy as sp
import math
from sympy import symbols, cos ,log, diff


In [192]:
def fixed_point_iteration(g, x0, tol=1e-7, max_iter=100):
    """
    Método do Ponto Fixo para encontrar a raiz de f(x) = 0,
    usando uma função g(x) tal que x = g(x).

    :param g: Função g(x) usada para a iteração.
    :param x0: Valor inicial (chute).
    :param tol: Tolerância para o erro.
    :param max_iter: Número máximo de iterações.
    :return: A raiz aproximada, número de iterações e lista de aproximações.
    """
    iter_count = 0
    x = x0
    values = [x0]

    while iter_count < max_iter:
        x_next = g(x)
        values.append(x_next)

        if abs(x_next - x) < tol:
            return x_next, iter_count + 1, values

        x = x_next
        iter_count += 1

    raise Exception("O método não convergiu dentro do número máximo de iterações.")


In [3]:
def plot_function(f, x_range):
    """
    Plota a função f(x) em um intervalo x_range.
    """
    x_vals = np.linspace(x_range[0], x_range[1], 400)
    y_vals = f(x_vals)

    # Cria o gráfico da função
    fig_fx = go.Figure()
    fig_fx.add_trace(go.Scatter(x=x_vals, y=y_vals,
                                mode='lines',
                                name='f(x) = x² + x - 6',
                                line=dict(color='darkblue', width=2)))

    fig_fx.show()

In [4]:
def fpi_print_history(g, x0):
    try:
        root, iters, history = fixed_point_iteration(g, x0)
        print(f"Raiz aproximada: {root}")
        print(f"Número de iterações: {iters}")
        print("Histórico de aproximações:")
        for i, val in enumerate(history):
            print(f"  Iteração {i}: {val}")
        return root, iters, history
    except Exception as e:
        print(f"Erro: {e}")
        return None, None, None

In [5]:
def fpi_plot_history(history):
    try:
        fig = px.line(
            x=list(range(len(history))),
            y=history,
            markers=True,
            labels={'x': 'Iteração', 'y': 'Aproximação'},
            title='Histórico de Aproximações - Método do Ponto Fixo',
        )
        fig.update_traces(
            marker=dict(size=10, color='red', line=dict(width=2, color='DarkSlateGrey')),
            line=dict(width=2, color='blue')
        )
        fig.show()
    except Exception as e:
        print("Erro ao plotar o gráfico:", e)
        return None

In [132]:
def fpi_plot_graph(f, history, root, g_label="g(x)"):
    """
    Plota f(x), as aproximações do método do ponto fixo e destaca a raiz aproximada.
    """
    if history is None or root is None:
        print("Histórico ou raiz não definidos.")
        return

    x_vals = np.linspace(min(history) - 1, max(history) + 1, 4000)
    y_vals = f(x_vals)

    fig_fx = go.Figure()
    fig_fx.add_trace(go.Scatter(x=x_vals, y=y_vals,
                                mode='lines',   
                                name='f(x)',
                                line=dict(color='darkblue', width=2)))

    fig_fx.add_trace(go.Scatter(x=history, y=[f(x) for x in history], 
                                mode='markers+lines', 
                                name=g_label,
                                marker=dict(size=7, color='red', line=dict(width=2, color='DarkSlateGrey'))))

    fig_fx.add_trace(go.Scatter(x=[root], y=[f(root)], mode='markers', name='Raiz Aproximada',
                                marker=dict(size=10, color='orange', symbol='cross', line=dict(width=2, color='Black'))))

    fig_fx.update_layout(title='f(x) e Aproximações pelo Método do Ponto Fixo',
                        xaxis_title='x',
                        yaxis_title='f(x)',
                        showlegend=True)
    fig_fx.show()


In [99]:
def fpi_plot_analysis(x_vals, g_vals,f = lambda x: x**2):
    """
    Generates a plot for graphical analysis of fixed points in the fixed-point iteration method.

    Parameters:
    - x_vals (array-like): Values of x in the domain of the function.
    - g_vals (array-like): Values of the function g(x) corresponding to x_vals.

    The plot includes:
    - The function f(x) = x^2 + x - 6.
    - The function g(x).
    - The identity line y = x (fixed points line).
    - The numerical derivative g'(x).
    - Reference lines at y = 1 and y = -1 for convergence analysis.

    Returns:
    - Interactively displays the plot.
    """

    

    if len(x_vals) != len(g_vals):
        raise ValueError("x_vals e g_vals devem ter o mesmo comprimento.")

    g_deriv = np.gradient(g_vals, x_vals)

    fig = go.Figure()

    # Plot f(x)
    y_vals = f(x_vals)
    fig.add_trace(go.Scatter(
        x=x_vals, y=y_vals,
        mode='lines',
        name='f(x)',
        line=dict(color='purple', width=2)
    ))

    # Plot g(x)
    fig.add_trace(go.Scatter(
        x=x_vals, y=g_vals,
        mode='lines',
        name='g(x)',
        line=dict(color='blue')
    ))

    # Plot y = x
    fig.add_trace(go.Scatter(
        x=x_vals, y=x_vals,
        mode='lines',
        name='y = x',
        line=dict(dash='dash', color='black')
    ))

    # Plot g'(x)
    fig.add_trace(go.Scatter(
        x=x_vals, y=g_deriv,
        mode='lines',
        name="g'(x)",
        line=dict(dash='dot', color='green')
    ))

    # Plot y = -1 and y = 1
    for y_ref in [-1, 1]:
        fig.add_trace(go.Scatter(
            x=[x_vals[0], x_vals[-1]],
            y=[y_ref, y_ref],
            mode='lines',
            name=f'y = {y_ref}',
            line=dict(color='gray', dash='dot')
        ))
    # Plot y = 0
    for y_ref in [0]:
        fig.add_trace(go.Scatter(
            x=[x_vals[0], x_vals[-1]],
            y=[y_ref, y_ref],
            mode='lines',
            name=f'y = {y_ref}',
            line=dict(color='gray')
        ))

    # # Layout
    y_min = min(-1, np.min(g_vals), np.min(g_deriv))
    y_max = max(1, np.max(g_vals), np.max(g_deriv))

    fig.update_layout(
        title='Graphical Analysis of Fixed Points',
        xaxis_title='x',
        yaxis_title='y',
        legend_title='Functions',
        showlegend=True,
        xaxis=dict(
            title='x',
            tickmode='linear'
        ),
        yaxis=dict(
            title='y',
            range=[y_min, y_max],
            tickmode='linear',
            dtick=0.5
        )
    )
    return fig


In [8]:
def symbolic_derivative(g):
    x = symbols('x')
    g_sym = g(x)
    g_prime = diff(g_sym, x)
    print("Symbolic derivative:", g_prime)
    return g_prime

## teste

## f(x) = x^2 + x - 6
```python

In [47]:
f = lambda x: x**2 + x - 6

```markdown
g(x) = sqrt(6-x)

In [48]:
g = lambda x: math.sqrt(6 - x)  # g(x) = sqrt(6 - x)

In [49]:
x0 = -0.2

_r, _i, history = fpi_print_history(g, x0)

Raiz aproximada: 1.999999880115105
Número de iterações: 12
Histórico de aproximações:
  Iteração 0: -0.2
  Iteração 1: 2.4899799195977463
  Iteração 2: 1.8735047585747557
  Iteração 3: 2.0313776708001012
  Iteração 4: 1.9921401379420822
  Iteração 5: 2.001964001189312
  Iteração 6: 1.999508939417548
  Iteração 7: 2.000122761378024
  Iteração 8: 1.9999693094200162
  Iteração 9: 2.0000076726302787
  Iteração 10: 1.9999980818415104
  Iteração 11: 2.000000479539565
  Iteração 12: 1.999999880115105


In [50]:
fpi_plot_history(history)

In [51]:
fpi_plot_graph(f, history, _r, g_label="g(x) = sqrt(6 - x)")

In [52]:
symbolic_derivative(g = lambda x: sp.sqrt(6 - x))

Symbolic derivative: -1/(2*sqrt(6 - x))


-1/(2*sqrt(6 - x))

In [101]:
g = lambda x: np.sqrt(6 - x)  # g(x) = sqrt(6 - x)
x_vals = np.linspace(min(history) - 20, max(history) + 20, 4000)
g_vals = g(x_vals)

fpi_plot_analysis(
    x_vals=x_vals,
    g_vals=g_vals,
    f=f
)

C:\Users\maria\AppData\Local\Temp\ipykernel_35580\3904308631.py:1: RuntimeWarning:

invalid value encountered in sqrt



## math domain error
```python

### ocorre quando alguma iteração de uma função matemática não é válida, nesse caso a raiz quadrada de um número negativo
```python

### exemplo: g(7)
sqrt(6-7) = sqrt(-1) = math domain error
``` 

In [ ]:
g = lambda x: math.sqrt(6-x)
x0 = 7

try:
    root, iters, history = fixed_point_iteration(g, x0)
    print(f"Raiz aproximada: {root}")
    print(f"Número de iterações: {iters}")
    print("Histórico de aproximações:")
    for i, val in enumerate(history):
        print(f"  Iteração {i}: {val}")
except Exception as e:
    print(e)

math domain error


### exemplo: g(-31)
a função g(-31) gera um erro, mas não na primeira iteração, e nas próximas.
```python
``` 

In [ ]:
g = lambda x: math.sqrt(6-x)
x0 = -31

try:
    root, iters, history = fixed_point_iteration(g, x0)
    print(f"Raiz aproximada: {root}")
    print(f"Número de iterações: {iters}")
    print("Histórico de aproximações:")
    for i, val in enumerate(history):
        print(f"  Iteração {i}: {val}")
except Exception as e:
    print(e,e.__doc__)

math domain error Inappropriate argument value (of correct type).


In [ ]:
g = lambda x: math.sqrt(6-x)
w1 = g(-31)
w1

6.082762530298219

In [ ]:
try:
    g(w1)
except Exception as e:
    print("Erro ao calcular g(w1):", e,e.__doc__)

Erro ao calcular g(w1): math domain error Inappropriate argument value (of correct type).


### essa função de iteração só pode ser implementada para valores contidos no intervalo [-30, 6]
```python
``` 

# exemplo 11

```python
``` 

In [ ]:
g = lambda x: math.sqrt(6-x)
x0 = -31

try:
    root, iters, history = fixed_point_iteration(g, x0)
    print(f"Raiz aproximada: {root}")
    print(f"Número de iterações: {iters}")
    print("Histórico de aproximações:")
    for i, val in enumerate(history):
        print(f"  Iteração {i}: {val}")
except Exception as e:
    print(e,e.__doc__)

# cos(ln(x))-x/2
```python
``` 

In [189]:
f = lambda x: np.cos(np.log(x)) - x/2

```markdown
g(x) = sqrt(6-x)

In [193]:
g = lambda x: np.cos(np.log(x)) * 2

In [194]:
x0 = 4

_r, _i, history = fpi_print_history(g, x0)

Raiz aproximada: 1.7156382007112252
Número de iterações: 34
Histórico de aproximações:
  Iteração 0: 4
  Iteração 1: 0.36691394948660344
  Iteração 2: 1.0761782343583912
  Iteração 3: 1.9946124977432524
  Iteração 4: 1.5419192518064584
  Iteração 5: 1.8153986768300758
  Iteração 6: 1.6548325472874716
  Iteração 7: 1.7516055388684537
  Iteração 8: 1.6939439302628625
  Iteração 9: 1.7285797213903225
  Iteração 10: 1.7078652316726088
  Iteração 11: 1.7202882130466568
  Iteração 12: 1.7128497477869598
  Iteração 13: 1.7173080132695542
  Iteração 14: 1.7146374849264123
  Iteração 15: 1.7162377068981072
  Iteração 16: 1.7152790288882682
  Iteração 17: 1.7158534358291593
  Iteração 18: 1.7155092966893812
  Iteração 19: 1.7157154868631388
  Iteração 20: 1.7155919518172957
  Iteração 21: 1.7156659667582752
  Iteração 22: 1.7156216217826656
  Iteração 23: 1.7156481905836816
  Iteração 24: 1.715632272235977
  Iteração 25: 1.715641809524458
  Iteração 26: 1.7156360953787386
  Iteração 27: 1.715639

In [195]:
fpi_plot_history(history)

In [196]:
fpi_plot_graph(f, history, _r, g_label="g(x) = np.cos(np.log(x)) * 2")

C:\Users\maria\AppData\Local\Temp\ipykernel_35580\2409415758.py:1: RuntimeWarning:

invalid value encountered in log



In [197]:
symbolic_derivative(g = lambda x: 2 * sp.cos(sp.log(x)))

Symbolic derivative: -2*sin(log(x))/x


-2*sin(log(x))/x

In [198]:

x_vals = np.linspace(min(history) - 2, max(history) + 2, 40000)
g_vals = g(x_vals)

fpi_plot_analysis(
    x_vals=x_vals,
    g_vals=g_vals,
    f=f
)

C:\Users\maria\AppData\Local\Temp\ipykernel_35580\1277808751.py:1: RuntimeWarning:

invalid value encountered in log

